In [4]:
!pip install langchain chromadb langchain-community huggingface-hub sentence-transformers langchain-groq -q


In [2]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 32.3 MB/s eta 0:00:00


In [3]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import PromptTemplate

from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings  # or OpenAIEmbeddings if you prefer
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain_groq import ChatGroq
import os
import re

In [39]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [26]:
GROQ_API_KEY='Your-API-Key'

In [27]:
def extract_entity_from_question(question: str) -> str:
    match = re.search(r"(?:in|on|about|of|for|into)\s+([A-Za-z]+)", question, re.IGNORECASE)
    return match.group(1).capitalize() if match else question.strip()

In [28]:
def load_documents_for_entity(entity: str):
    urls = [
        f"https://www.screener.in/company/{entity.upper()}/",
        f"https://finance.yahoo.com/quote/{entity.upper()}/",
        f"https://www.moneycontrol.com/financials/{entity.lower()}/",
    ]
    loader = WebBaseLoader(urls)
    docs = loader.load()
    return docs

In [40]:
def create_vectorstore(documents):
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = splitter.split_documents(documents)

    # Use local or OpenAI embedding
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = Chroma.from_documents(chunks, embedding=embeddings, persist_directory="./chroma_db")
    vectorstore.persist()

    return vectorstore

In [41]:
def create_investment_chain(retriever):
    llm = ChatGroq(
        temperature=0.2,
        model_name="llama3-8b-8192",
        groq_api_key=GROQ_API_KEY
    )

    prompt = PromptTemplate.from_template(
        """You are a top-tier financial analyst. Based on the following information from financial websites, answer the user's question.

        Context:
        --------
        {context}

        Question:
        ---------
        {input}

        Your Answer (with reasoning and suggestion):"""
    )

    document_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(retriever, document_chain)

    return rag_chain

In [42]:
def analyze_investment_question(question: str):
    entity = extract_entity_from_question(question)
    print(f"[INFO] Extracted Entity: {entity}")

    docs = load_documents_for_entity(entity)
    vectorstore = create_vectorstore(docs)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

    chain = create_investment_chain(retriever)
    result = chain.invoke({"input": question})

    return result["answer"]

In [44]:
question = "Should I currently invest in HDFC Bank?"
answer = analyze_investment_question(question)
print(answer)

[INFO] Extracted Entity: Hdfc
Based on the provided information, I would not recommend investing in HDFC Bank at this time. Here's a breakdown of the reasons:

1. **Stock Price**: HDFC Bank's current price is ₹2,724, which is trading at 3.73 times its book value. This is relatively high, indicating that the stock may be overvalued.
2. **Sales Growth**: The company has delivered a poor sales growth of 8.14% over the past five years, which is a concern.
3. **Return on Equity (ROE)**: HDFC Bank's ROE is 12.8%, which is relatively low compared to its peers.
4. **Interest Coverage Ratio**: The company has a low interest coverage ratio, which indicates that it may struggle to meet its interest payments.
5. **Dividend Yield**: The dividend yield is 0.00%, which is extremely low and may not provide a sufficient return for investors.

While HDFC Bank has some positive aspects, such as its pioneer status in housing finance and a healthy dividend payout, the concerns mentioned above outweigh the 